In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir traindir
!mkdir testdir

mkdir: cannot create directory ‘traindir’: File exists
mkdir: cannot create directory ‘testdir’: File exists


In [3]:
!tar -xzf "/content/drive/MyDrive/HW4/UTKFace.tar.gz" --directory "/content/traindir" --checkpoint=.100
!tar -xzf "/content/drive/MyDrive/HW4/crop_part1.tar.gz" --directory "/content/testdir" --checkpoint=.100

...............................................................................................................................................................................................

## Analyzing Fairness of the Model
**NOTE:** Feel free to change the code template. 

In [4]:
import cv2
import os
import numpy as np

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [5]:
def load_dataset(path, use_less_size=False):
    data = []
    labels = []
    images = os.listdir(path)
    for image_i in range(0, 5000 if use_less_size else len(images)):
        image = images[image_i]
        img_path = os.path.join(path, image)
        try:
          if img_path.endswith('.jpg'):
              img = cv2.imread(img_path)
              img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
              img = cv2.resize(img, (299, 299)) # Resize the image to 299x299
              label = int(image.split('_')[2])
              data.append(img)
              labels.append(label)
        except ValueError:
          print()
    return np.array(data), np.array(labels)

def split_train_test(X_train, y_train):
  test_size = 0.3
  return train_test_split(X_train, y_train, test_size = test_size, train_size = 1 - test_size, random_state=42)

def load_model():
  return InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [6]:
#Note: Replace "/path/to/UTKFace" with the actual path to the UTKFace dataset on your local machine.
X_train, y_train = load_dataset('/content/traindir/UTKFace')

X_train = X_train.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)

X_train, X_val, y_train, y_val = split_train_test(X_train, y_train)

base_model = load_model()
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#TODO: Define your hyperparameters
loss_func = "categorical_crossentropy"
optim = "adam"
batch_size = 15
epochs = 10

model.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))




Epoch 1/10
1107/1107 [==============================] - 123s 74ms/step - loss: 1.3374 - accuracy: 0.4815 - val_loss: 1.2346 - val_accuracy: 0.5336
Epoch 2/10
1107/1107 [==============================] - 71s 64ms/step - loss: 1.1324 - accuracy: 0.5625 - val_loss: 1.4138 - val_accuracy: 0.4473
Epoch 3/10
1107/1107 [==============================] - 71s 64ms/step - loss: 0.8578 - accuracy: 0.7008 - val_loss: 3.5188 - val_accuracy: 0.4290
Epoch 4/10
1107/1107 [==============================] - 71s 64ms/step - loss: 0.7429 - accuracy: 0.7422 - val_loss: 0.8394 - val_accuracy: 0.7019
Epoch 5/10
1107/1107 [==============================] - 71s 64ms/step - loss: 0.6767 - accuracy: 0.7688 - val_loss: 0.7394 - val_accuracy: 0.7452
Epoch 6/10
1107/1107 [==============================] - 71s 64ms/step - loss: 0.6242 - accuracy: 0.7861 - val_loss: 0.6730 - val_accuracy: 0.7790
Epoch 7/10
1107/1107 [==============================] - 71s 64ms/step - loss: 0.5836 - accuracy: 0.8017 - val_loss: 0.65

In [7]:
# Load the validation set and preprocess it
X_val, y_val = load_dataset('/content/testdir/crop_part1', use_less_size = True)

X_val = X_val.astype('float32') / 255.0
y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

# Make predictions using the trained model
y_pred = model.predict(X_val)

# Convert the one-hot encoded vectors to integer labels
y_val_labels = np.argmax(y_val, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

# Initialize an empty dictionary to store the results
results_dict = {}

# Compute the number of positive/negative predictions for each race and store in the dictionary
races = ['White', 'Black', 'Asian', 'Indian', 'Others']
for i, race in enumerate(races):
    race_indices = np.where(y_val_labels == i)[0]
    race_predictions = y_pred_labels[race_indices]

    other_race_indices = np.where(y_val_labels != i)[0]
    other_race_predictions = y_pred_labels[other_race_indices]

    num_true_positives = np.sum(race_predictions == i)
    num_false_negative = np.sum(race_predictions != i)

    num_false_positive = np.sum(other_race_predictions == i)
    num_true_negative = np.sum(other_race_predictions != i)

    # num_negatives = len(race_indices) - num_positives
    results_dict[race] = {'TP': num_true_positives, 'TN': num_true_negative, 'FP': num_false_positive, 'FN': num_false_negative}

results_dict



157/157 [==============================] - 6s 25ms/step


{'White': {'TP': 2646, 'TN': 1840, 'FP': 449, 'FN': 63},
 'Black': {'TP': 177, 'TN': 4727, 'FP': 73, 'FN': 21},
 'Asian': {'TP': 705, 'TN': 4148, 'FP': 50, 'FN': 95},
 'Indian': {'TP': 546, 'TN': 4174, 'FP': 112, 'FN': 166},
 'Others': {'TP': 157, 'TN': 4336, 'FP': 83, 'FN': 422}}

In [8]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val_labels, y_pred_labels)

array([[2646,    9,   17,   17,   20],
       [  11,  177,    0,    9,    1],
       [  54,    8,  705,   13,   20],
       [  94,   24,    6,  546,   42],
       [ 290,   32,   27,   73,  157]])

In [9]:
def compare_metrics_across_races(race1, race2, metric):
    if metric == 'statistical_parity':
        p1 = results_dict[race1]['TP'] / (results_dict[race1]['TP'] + results_dict[race1]['FN'])
        p2 = results_dict[race2]['TP'] / (results_dict[race2]['TP'] + results_dict[race2]['FN'])
        return abs(p1 - p2)

    elif metric == 'equalized_odds':
        tpr1 = results_dict[race1]['TP'] / (results_dict[race1]['TP'] + results_dict[race1]['FN'])
        tpr2 = results_dict[race2]['TP'] / (results_dict[race2]['TP'] + results_dict[race2]['FN'])
        fpr1 = results_dict[race1]['FP'] / (results_dict[race1]['FP'] + results_dict[race1]['TN'])
        fpr2 = results_dict[race2]['FP'] / (results_dict[race2]['FP'] + results_dict[race2]['TN'])
        return abs(tpr1 - tpr2), abs(fpr1 - fpr2)

    elif metric == 'calibration':
        cal1 = results_dict[race1]['TP'] / (results_dict[race1]['TP'] + results_dict[race1]['FP'])
        cal2 = results_dict[race2]['TP'] / (results_dict[race2]['TP'] + results_dict[race2]['FP'])
        return abs(cal1 - cal2)

already_compared = []
for id, race1 in enumerate(races):
  for id1, race2 in enumerate(races[1:][id:]):
      print(f"Comparison between {race1} and {race2}:")
      eo_diff_tpr, eo_diff_fpr = compare_metrics_across_races(race1, race2, 'equalized_odds')
      print(f"Statistical Parity: {compare_metrics_across_races(race1, race2, 'statistical_parity'):.4f}")
      print(f"Equalized Odds (tpr, fpr): ({eo_diff_tpr:.4f}, {eo_diff_fpr:.4f})")
      print(f"Calibration: {compare_metrics_across_races(race1, race2, 'calibration'):.4f}")
      print()

Comparison between White and Black:
Statistical Parity: 0.0828
Equalized Odds (tpr, fpr): (0.0828, 0.1809)
Calibration: 0.1469

Comparison between White and Asian:
Statistical Parity: 0.0955
Equalized Odds (tpr, fpr): (0.0955, 0.1842)
Calibration: 0.0788

Comparison between White and Indian:
Statistical Parity: 0.2099
Equalized Odds (tpr, fpr): (0.2099, 0.1700)
Calibration: 0.0251

Comparison between White and Others:
Statistical Parity: 0.7056
Equalized Odds (tpr, fpr): (0.7056, 0.1774)
Calibration: 0.2008

Comparison between Black and Asian:
Statistical Parity: 0.0127
Equalized Odds (tpr, fpr): (0.0127, 0.0033)
Calibration: 0.2258

Comparison between Black and Indian:
Statistical Parity: 0.1271
Equalized Odds (tpr, fpr): (0.1271, 0.0109)
Calibration: 0.1218

Comparison between Black and Others:
Statistical Parity: 0.6228
Equalized Odds (tpr, fpr): (0.6228, 0.0036)
Calibration: 0.0538

Comparison between Asian and Indian:
Statistical Parity: 0.1144
Equalized Odds (tpr, fpr): (0.1144, 

These predictions are for smaller (5000) test gorup as unable to produce complete results due to resources constrainsts.

**Statistical Parity:**
There are significant differences in predicted outcomes between some subgroups, such as Blacks vs Asians or Blacks vs Indians, compared to the overall population, whereas they are relatively small in other pairs, such as Whites vs Blacks or Whites vs Indians.

**Equalized Odds:**
Based on a machine learning model with equalized odds fairness constraint, these results illustrate the differences in true positive (or false positive) rates between different races. For each pair, a range of values is presented to determine the acceptable range of true positive (or false positive) rates to meet the equalized odds constraint.

**Calibration:** For calibration the model has slight changes for each subgroups, White and Indians has relatively high compared to the overall population, which is relatively small in other groups.


- There is a significant difference between the model's predictions for white and black applicants, as the model overestimates probability for Black applicants by positive outcomes.

- In the case of Asian applicants, the expected probabilities are only of positive results.

- In the case of Indian applicants, the model is underconfident, since the predicted probabilities what is observed in the subgroup.

- In the case of Other applicants, the model is slightly overconfident, as the predicted probabilities is higher than observed positive outcomes.

- Compared to black applicants, the model predicts probabilities of positive outcomes is greater than the observed frequency of positive outcomes for Asian applicants.

- Model predictions for Indian applicants is greater than the observed positive outcomes for that subgroup, which indicates a moderate overconfidence.

- Models for Black applicants are overconfident, as predicted probabilities are observed positive outcome rates for Other applicants.

- The model underestimates the likelihood of a positive outcome for Indian applicants, with predicted probabilities of more than observed frequencies.

- Compared to Asian applicants, the model is a little underconfident, since the predicted probabilities are only more than actual positive outcomes.

- For Other applicants, the predicted probability of positive outcomes is greater than that observed for Indian applicants.